In [1]:
# IMPORTS
import os
import json
import time

from tqdm import tqdm
import requests
import geopandas as gpd
import pandas as pd

In [2]:
# SIGN IN

url = "http://localhost:8000"
token = ""

# TODO: DO NOT PUSH THESE!!!!!!!!!!!!!!!!!!!!!!
sign_in = requests.post(
    url=f"{url}/users/authtoken/",
    data={
        "username": "",
        "password": ""
    }
)
if sign_in.status_code == 200:
    token = sign_in.json()["login"]["token"]
    headers = {"Authorization": f"Token {token}", "Content-type": "application/json"}

    print(sign_in.status_code)
else:
    print(f"Issue signing in\n {sign_in.json()}")

200


In [20]:
# GET/UPDATE FIELD
wacky_field = gpd.read_file("fixed_wacky_field.geojson")

wacky_field = json.loads(wacky_field.to_json())
del wacky_field["features"][0]["id"]
get_field = requests.put(
    url=f"{url}/layers/getupdatedeletelayer/a50b7391-dcce-494e-b022-844c36bb83ff/",
    headers=headers, data=json.dumps(wacky_field["features"][0])
)
get_field
# wacky_field["features"][0]

<Response [200]>

In [16]:
# UPLOAD AND FORMAT NEW POLYGONS

gdf = gpd.read_file("corrected_coordinate_system_farmafrica_new.geojson")

# gdf.columns
# delete_columns = [
#     'id', 'elevation', 'name', 'comment', 'description', 'url', 'url_name',
#     'symbol', 'source', 'type', 'feature_type', 'file_name',
#     'cmt', 'desc', 'src', 'link1_href', 'link1_text', 'link1_type',
#     'link2_href', 'link2_text', 'link2_type', 'number',
#     'gpxx_TrackExtension', 'layer', 'path'
# ]
# gdf["pyxus_id"] = gdf["id"]
# gdf.drop(delete_columns, inplace=True, axis=1)
gdf['user_id'] = ""
new_fields = json.loads(gdf.to_json())
print(f"{len(new_fields['features'])} is the length of the new fields")

gdf.head()
# new_fields["features"][0]
# gdf.iloc[0]
# gdf.loc[gdf["field_id"] == "a0299eaa-83e7-4a26-924c-3c14ebced544"]


14 is the length of the new fields


,Crop type,Farm name(,Acreage (a,field_id,geometry,user_id
0,Sorghum,Harieth Jocktan Mwendi,1.0,916d55e0-8466-4c07-9e25-61acea2aa21d,"POLYGON ((36.83080 -6.31170, 36.83080 -6.31160...",5f980108050537000a61fa8c
1,Sorghum,Timotheo Elias Kusenha,1.0,b79a0e85-0077-4fd3-a5b8-b4b09ae6c054,"POLYGON ((36.82520 -6.31640, 36.82520 -6.31630...",5f980108050537000a61fa8c
2,Sorghum,Loty Philimon Malekela,1.0,8a7eb6ae-9589-4b91-b5a2-1640dc7d47f5,"POLYGON ((36.82630 -6.31640, 36.82630 -6.31630...",5f980108050537000a61fa8c
3,Sorghum,Donatela Joseph Gailanga,5.0,291fe995-d405-47d6-8bd7-8e183b65e620,"POLYGON ((36.37550 -6.59870, 36.37550 -6.59860...",5f980108050537000a61fa8c
4,Sorghum,Sevelina Albino Kuzenjeza,7.0,d4a52e32-9fae-4687-8d46-ed1d4506c7c1,"POLYGON ((36.38550 -6.62750, 36.38550 -6.62750...",5f980108050537000a61fa8c


In [17]:
# UPLOAD NEW POLYGONS LOCALLY

if sign_in.status_code == 200:
    #########delete all known rows ###########
    # for o_field in tqdm(existing_fields["features"]):
    #     delete_field = requests.delete(
    #         url=f"{url}/layers/getupdatedeletelayer/{o_field['properties']['field_id']}/",
    #         headers=headers
    #     )
    ########################################
    for n_field in tqdm(new_fields["features"]):
        # existing_row = list(filter(
        #     lambda field: field["properties"]["field_id"] == n_field["properties"]["field_id"],
        #     existing_fields["features"]
        # ))
        # if existing_row:
        #     existing_row = existing_row[0]
        #     try:
        #         del existing_row["id"]
        #         existing_row["properties"]["field_attributes"]["layer"] = n_field["properties"]["layer"]
        #         # delete_field = requests.delete(
        #         #     url=f"{url}/layers/getupdatedeletelayer/{existing_row['properties']['field_id']}/",
        #         #     headers=headers
        #         # )
        #         # update_field = requests.put(
        #         #     url=f"{url}/layers/getupdatedeletelayer/{existing_row['properties']['field_id']}/",
        #         #     data=json.dumps(existing_row), headers=headers
        #         #     # this affects the frontend part as well. Tested it.
        #         # )
        #         time.sleep(0.5)
        #     except KeyError:
        #         # print(existing_row.keys(), existing_row["properties"]["field_id"])
        #         continue
        # if not existing_row:
        n_field["properties"]["field_attributes"] = {}
        n_field["properties"]["field_attributes"]["CropType"] = n_field["properties"]["Crop type"]
        del n_field["properties"]["Crop type"]

        n_field["properties"]["field_attributes"]["Farm name("] = n_field["properties"]["Farm name("]
        del n_field["properties"]["Farm name("]

        n_field["properties"]["field_attributes"]["Acreage (a"] = n_field["properties"]["Acreage (a"]
        del n_field["properties"]["Acreage (a"]
        update_field = requests.post(
            url=f"{url}/layers/listcreatepolygonlayer/",
            data=json.dumps(n_field), headers=headers
        )
        # time.sleep(0.5)
else:
    print(f"Failed to sign in. Error is\n\n{sign_in.json()}")

100%|██████████| 14/14 [00:29<00:00,  2.12s/it]


In [ ]:
# UPLOAD NEW POLYGONS HOSTEDLINE

bulk_upload = {}
for n_field in tqdm(new_fields["features"]):
    n_field["properties"]["field_attributes"] = {}
    n_field["properties"]["field_attributes"]["layer"] = n_field["properties"]["layer"]
    del n_field["properties"]["layer"]
    n_field["properties"]["field_attributes"]["CropType"] = "Tobacco"
    bulk_upload[n_field["properties"]["field_id"]] = n_field
update_field = requests.post(
    url=f"{url}/layers/listcreatepolygonlayer/",
    data=json.dumps(bulk_upload), headers=headers
)

In [12]:
# GET OLD POLYGONS

if sign_in.status_code == 200:
    existing_fields = requests.get(f"{url}/layers/listcreatepolygonlayer/", headers=headers)
    existing_fields = existing_fields.json()
    print(len(existing_fields["features"]), "is the length of the old fields\n")
else:
    print(f"Failed to sign in. Error is\n\n{sign_in.json()}")

1058 is the length of the old fields



In [13]:
# TRIAL
# len(existing_fields["features"])
gdf = gpd.read_file("fixed_single_pyxus_data_16042021.geojson")
json_gdf = json.loads(gdf.to_json())

new_pyxus_fields = []
found_rows = 0
for new_row in json_gdf["features"]:
    existing_row = list(filter(
        lambda field: field["properties"]["field_id"] == new_row["properties"]["field_id"],
        existing_fields["features"]
    ))
    if existing_row:
        found_rows += 1
    elif not existing_row:
        new_pyxus_fields.append(new_row)

print(
    f"New row length is {len(new_pyxus_fields)} and old fields are {found_rows}"
)


New row length is 0 and old fields are 4278


In [48]:
# DELETE ALL EXISTING ROWS

#########delete all known rows ###########
# for o_field in tqdm(existing_fields["features"]):
#     delete_field = requests.delete(
#         url=f"{url}/layers/getupdatedeletelayer/{o_field['properties']['field_id']}/",
#         headers=headers
#     )
########################################

In [28]:
# REMOVE OLD GRIDS


old_grids = requests.get(f"{url}/layers/gridlayers/", headers=headers)


if old_grids.status_code == 200:
    old_grids = old_grids.json()
    # with open('check_old_grids.json', 'w') as f:
    #     json.dump(old_grids, f)
    for o_grid in tqdm(old_grids["features"]):
        delete_o_grid = requests.delete(
            url=f"{url}/layers/gridlayers/{o_grid['properties']['field_id']}/",
            headers=headers
        )
        time.sleep(0.5)

100%|██████████| 1/1 [00:00<00:00,  1.93it/s]


In [52]:
# UPLOAD NEW GRIDS

with open('new_oppo_grids.json') as f:
    new_grids = json.load(f)
    # for n_grid in tqdm(new_grids["features"]):
    #     n_grid["properties"]["user_id"] = ""
    #     # n_grid["properties"]["field_attributes"] = json.loads(
    #     #     n_grid["properties"]["field_attributes"]
    #     # )
    #     create_grid = requests.post(
    #         url=f"{url}/layers/gridlayers/",
    #         data=json.dumps(n_grid), headers=headers
    #     )
    #     time.sleep(0.5)

    ########## UPLOADING ONLINE #############
    bulk_grids = {}
    for n_grid in tqdm(new_grids["features"]):
        n_grid["properties"]["user_id"] = ""
        # n_grid["properties"]["field_attributes"] = json.loads(
        #     n_grid["properties"]["field_attributes"]
        # )
        # del n_grid["properties"]["id"]
        bulk_grids[n_grid["properties"]["field_id"]] = n_grid

    create_grid = requests.post(
        url=f"{url}/layers/gridlayers/",
        data=json.dumps(bulk_grids), headers=headers
    )

100%|██████████| 100/100 [00:00<00:00, 334474.00it/s]


KeyboardInterrupt: 